# Experiment Notebook - New Test
This notebook will illustrate how to create and run a new experiment, or Test Run.

In [1]:
import sqlite3
# Create a connection to the database
db_connection = sqlite3.connect('experiment.db')


## - Creating a new Test Run
Each experiment is represented by a Test Run, which will consume a Dataset to perform completions on a related set of Questions.

The generated Responses and their Contexts will then be evaluated by the configured Test Eval functions.

In [2]:
# Create a new Test Run
from eval_data.models.testrun import TestRunModel, TestRunType
from eval_data.models.datasource import DatasourceModel

datasource = DatasourceModel(db_connection).get_datasource_by_name("BioASQ")

test_run = TestRunModel(db_connection).add_or_get_test_run(TestRunType(datasource_id=3, description="Base test on BioASQ"))

print(f"Test Run ID: {test_run.id}")


Test Run ID: 1


## - Load document text index


In [3]:
from eval_data.models.document import DocumentModel
from typing import List

documents = DocumentModel(db_connection).get_documents_by_datasource(datasource_id=test_run.datasource_id)

# Get questions and answers
from eval_data.models.question import QuestionModel, QuestionType

questions: List[QuestionType] = []
for doc in documents:
    questions.extend(QuestionModel(db_connection).get_questions_by_document_id(document_id=doc.id))

# Build the document index
from eval_scripts.documents import load_documents

nodes = load_documents(documents)
print(f"Loaded {len(documents)} documents")
print(f"Loaded {len(nodes)} nodes")

/Users/space/Documents/projects/dr-eval/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/space/Documents/projects/dr-eval/venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


load_huggingface_document: rag-datasets/rag-mini-bioasq;text-corpus
collecting documents from rag-datasets/rag-mini-bioasq;text-corpus
loading document 0
loading document 1
loading document 2
loading document 3
loading document 4
loading document 5
loading document 6
loading document 7
loading document 8
loading document 9
loading document 10
loading document 11
loading document 12
loading document 13
loading document 14
loading document 15
loading document 16
loading document 17
loading document 18
loading document 19
loading document 20
loading document 21
loading document 22
loading document 23
loading document 24
loading document 25
loading document 26
loading document 27
loading document 28
loading document 29
loading document 30
loading document 31
loading document 32
loading document 33
loading document 34
loading document 35
loading document 36
loading document 37
loading document 38
loading document 39
loading document 40
loading document 41
loading document 42
loading documen

KeyboardInterrupt: 

## - Create instance of the LLM query engine

In [ ]:
from my_generator import build_query_engine

query_engine = build_query_engine(nodes)

## - Response Generation
Utilizing the RAG system to generate responses to the questions during test runs.

In [ ]:
from packages.data.src.eval_data.models.response import ResponseModel, ResponseType
from packages.data.src.eval_data.models.context import ContextModel, ContextType

# Generate responses
for q in questions:
    res = query_engine.query(q.question)
    response = ResponseType(
        test_run_id = test_run.id,
        question_id = q.id,
        response = res.response
    )
    response_id = ResponseModel(db_connection).add_response(response)
    print(f"Response created: {response_id} - len: {len(response.response)}")

    context_model = ContextModel(db_connection)
    for c in res.source_nodes:
        context_model.add_context(
            ContextType(
                response_id=response_id, 
                text=c.node.get_content(),
                similarity_score=c.score
            )
        )